In [1]:
import pandas as pd
import numpy as np
import cv2
import shutil
from glob import glob
import os
import matplotlib.pyplot as plt
from PIL import Image


In [2]:
### 설정해주세요 ###

# 테스트셋의 정답으로 삼을 csv 파일
df = pd.read_csv('/opt/ml/segmentation/semantic-segmentation-level2-cv-16/dev/model_develop/mmsegmentation/work_dirs/10-2/10-2_TTA_28000it_crf.csv')
data_root = '/opt/ml/segmentation/input/data' # 이 아래에 batch_01, batch_02, batch_03 폴더가 있어야 합니다

# 원래 있던 training 폴더를 복제해서 training_pseudo 폴더로 만들어주셔야 합니다
# image, annotation 모두!
image_path = os.path.join(*(data_root, 'mmseg', 'images_v1', 'training_pseudo'))
ann_pth = os.path.join(*(data_root, 'mmseg', 'annotations_v1', 'training_pseudo'))

annotations = sorted(glob(os.path.join(image_path, '*')), reverse=True)
images = sorted(glob(os.path.join(ann_pth, '*')), reverse=True)

last_id_from_images = int(images[0].split('.')[0].split('/')[-1])
last_id_from_annotations = int(annotations[0].split('.')[0].split('/')[-1])

assert last_id_from_images == last_id_from_annotations
id_start = last_id_from_images + 1



In [60]:
for i in range(len(df)):
    cur = df.iloc[i, :]
    old_path = os.path.join(data_root, cur['image_id'])
    new_path = os.path.join(image_path, str(id_start + i).zfill(4) + '.jpg')
    shutil.copyfile(old_path, new_path)
    
    arr = np.array(cur['PredictionString'].split()).reshape(256, 256)
    mask = Image.fromarray(np.uint8(arr)).convert('RGB')
    mask = mask.resize((512, 512), Image.LANCZOS)
    new_ann_path = os.path.join(ann_pth, str(id_start + i).zfill(4) + '.png')
    mask.save(new_ann_path)